### Imports

In [1]:
import sys
if not hasattr(sys.modules['__main__'], '__file__'):
    %run "C:\Users\james\Documents\MLB\Code\U1. Imports.ipynb"
    %run "C:\Users\james\Documents\MLB\Code\U2. Utilities.ipynb"
    %run "C:\Users\james\Documents\MLB\Code\U3. Classes.ipynb"
    %run "C:\Users\james\Documents\MLB\Code\U4. Datasets.ipynb"
    %run "C:\Users\james\Documents\MLB\Code\U5. Models.ipynb"
    print("Imports executed")

Imports executed


### Games

In [2]:
if not hasattr(sys.modules['__main__'], '__file__'):
    # Set date range 
    start_date = "20240328"
    end_date = yesterdaysdate
    all_game_df = pd.read_csv(os.path.join(baseball_path, "game_df.csv"))
    game_df = all_game_df[(all_game_df['date'].astype(str) >= start_date) & (all_game_df['date'].astype(str) <= end_date)].reset_index(drop=True)

### Data

Create PA model inputs

In [3]:
%%time
complete_dataset = pd.read_csv(os.path.join(baseball_path, "Final Dataset.csv"))

CPU times: total: 50.2 s
Wall time: 54.3 s


In [4]:
complete_dataset = complete_dataset.replace([float('inf'), float('-inf')], 0)

In [6]:
complete_dataset['date'].tail()

1698399    20250417.0
1698400    20250417.0
1698401    20250417.0
1698402    20250417.0
1698403           NaN
Name: date, dtype: float64

Read in Steamer

In [ ]:
%%time
steamer_hitters_df = pd.read_csv(os.path.join(baseball_path, "A03. Steamer", "steamer_hitters_weekly_log.csv"), encoding='iso-8859-1')
steamer_hitters_df_current = pd.read_csv(os.path.join(baseball_path, "A03. Steamer", "steamer_hitters.csv"), encoding='iso-8859-1')
steamer_hitters_df_current['proj_year'] = steamer_hitters_df_current['proj_season'].copy()
steamer_hitters_df = pd.concat([steamer_hitters_df, steamer_hitters_df_current], axis=0)

In [ ]:
%%time
steamer_pitchers_df = pd.read_csv(os.path.join(baseball_path, "A03. Steamer", "steamer_pitchers_weekly_log.csv"), encoding='iso-8859-1')
steamer_pitchers_df_current = pd.read_csv(os.path.join(baseball_path, "A03. Steamer", "steamer_pitchers.csv"), encoding='iso-8859-1')
steamer_pitchers_df_current['proj_year'] = steamer_pitchers_df_current['proj_season'].copy()
steamer_pitchers_df = pd.concat([steamer_pitchers_df, steamer_pitchers_df_current], axis=0)

### Clean

Clean Steamer

In [ ]:
# Hitters
steamer_hitters_df['proj_year'].fillna(2025, inplace=True) # Shouldn't happen
steamer_hitters_df['proj_date'].fillna(todaysdate_dash, inplace=True)
steamer_hitters_df = clean_steamer_hitters(steamer_hitters_df)

# Pitchers
steamer_pitchers_df['proj_year'].fillna(2025, inplace=True) # Shouldn't happen
steamer_pitchers_df['proj_date'].fillna(todaysdate_dash, inplace=True)
steamer_pitchers_df = clean_steamer_pitchers(steamer_pitchers_df)

Shrink for faster processing

In [ ]:
complete_dataset.drop(columns={'description', 'batterName', 'pitcherName', 'postOnFirst', 'postOnSecond', 'postOnThird', 
                               'preOnFirst', 'preOnSecond', 'preOnThird', 'pitch_name'}, inplace=True)

In [ ]:
complete_dataset = complete_dataset[complete_dataset['date'].astype(int) > game_df["date"].astype(int).min()-10000]
steamer_hitters_df = steamer_hitters_df[steamer_hitters_df['date'].astype(int) > game_df["date"].astype(int).min()-10000]
steamer_pitchers_df = steamer_pitchers_df[steamer_pitchers_df['date'].astype(int) > game_df["date"].astype(int).min()-10000]

Create vs subsets

In [ ]:
vs_lhp = complete_dataset[complete_dataset['pitchHand'] == "L"].copy()
vs_rhp = complete_dataset[complete_dataset['pitchHand'] == "R"].copy()
vs_lhp['id'] = vs_lhp['batter'].astype(str) # note, this won't match what's in id field already
vs_rhp['id'] = vs_rhp['batter'].astype(str)
vs_lhp.drop_duplicates(subset=['id', 'date'], keep='last', inplace=True)
vs_lhp.drop_duplicates(subset=['id', 'date'], keep='last', inplace=True)
vs_lhp['date_time'] = pd.to_datetime(vs_lhp['date'], format='%Y%m%d')
vs_rhp['date_time'] = pd.to_datetime(vs_rhp['date'], format='%Y%m%d')
vs_lhp.sort_values('date_time', inplace=True)
vs_rhp.sort_values('date_time', inplace=True)

vs_lhb = complete_dataset[complete_dataset['batSide'] == "L"].copy()
vs_rhb = complete_dataset[complete_dataset['batSide'] == "R"].copy()
vs_lhb['id'] = vs_lhb['pitcher'].astype(str)
vs_rhb['id'] = vs_rhb['pitcher'].astype(str)
vs_lhb.drop_duplicates(subset=['id', 'date'], keep='last', inplace=True)
vs_lhb.drop_duplicates(subset=['id', 'date'], keep='last', inplace=True)
vs_lhb['date_time'] = pd.to_datetime(vs_lhb['date'], format='%Y%m%d')
vs_rhb['date_time'] = pd.to_datetime(vs_rhb['date'], format='%Y%m%d')
vs_lhb.sort_values('date_time', inplace=True)
vs_rhb.sort_values('date_time', inplace=True)

Prep for merge_asof

In [ ]:
steamer_hitters_df['date_time'] = pd.to_datetime(steamer_hitters_df['date'], format='%Y%m%d')
steamer_pitchers_df['date_time'] = pd.to_datetime(steamer_pitchers_df['date'], format='%Y%m%d')
steamer_hitters_df.sort_values('date_time', inplace=True)
steamer_pitchers_df.sort_values('date_time', inplace=True)
steamer_hitters_df['id'] = steamer_hitters_df['mlbamid'].astype(int).astype(str)
steamer_pitchers_df['id'] = steamer_pitchers_df['mlbamid'].astype(int).astype(str)

### Functions

##### Create Matchup File

In [ ]:
def create_matchup_file(game_df, row, vs_lhp, vs_rhp, vs_rhb, vs_lhb, steamer_hitters_df, steamer_pitchers_df, team_dict):
    game_id = game_df['game_id'][row]
    game_datetime = game_df['game_datetime'][row]
    game_date = game_df['game_date'][row]
    date = int(game_date.replace("-", ""))
    away_id = game_df['away_id'][row]
    home_id = game_df['home_id'][row]
    
    # Retrieve Baseball Reference team abbreviation
    away_team = team_dict[away_id]
    home_team = team_dict[home_id]


    for team in away_team, home_team:
        # Read in rosters
        roster_df = pd.read_csv(os.path.join(baseball_path, "A05. Rosters", "2. Rosters", f"Rosters {date}", f"Roster {team} {date}.csv"), encoding='iso-8859-1', dtype='str')

        # Read in batting orders
        order_df = pd.read_csv(os.path.join(baseball_path, "A05. Rosters", "1. Batting Orders", f"Batting Orders {date}", f"Batting Order {team} {game_id}.csv"), encoding='iso-8859-1', dtype='str')        
        
        # Read in bullpens
        bullpen_df = pd.read_csv(os.path.join(baseball_path, "A04. Bullpens", f"Bullpens {date}", f"Bullpen {team} {date}.csv"), encoding='iso-8859-1', dtype='str')  
        bullpen_df['id'] = bullpen_df['id'].apply(lambda x: x.replace('.0', '') if isinstance(x, str) else x)
        
        # Drop if missing id
        bullpen_df = bullpen_df[(bullpen_df['id'].notna()) & (bullpen_df['id'] != "")].reset_index(drop=True)
        
        # Merge batting order onto roster
        team_df = pd.merge(roster_df, order_df[['id', 'fullName', 'position', 'status', 'order']], on='id', how='outer', suffixes=("","2"))
        
        # Fill in missings
        team_df['batSide'].fillna('Right', inplace=True)
        team_df['pitchHand'].fillna('Right', inplace=True)
        team_df['fullName'].fillna(team_df['fullName2'], inplace=True)
        team_df['position'].fillna(team_df['position2'], inplace=True)

        # Clean name
        team_df['fullName'] = team_df['fullName'].apply(remove_accents)
        
        # Merge pitcher leverage onto roster
        team_df = pd.merge(team_df, bullpen_df[['id', 'Leverage']], on=['id'], how='left')
        
        # Add weather
        box = create_box(game_id)
        team_df['weather'] = box[0]
        team_df['wind'] = box[1]
        team_df['park'] = box[2]
        team_df = clean_weather(team_df)

        # Add venue
        team_df['venue_id'] = game_df['venue_id'][row]
        
        # Add starters
        team_df['away_starter'] = game_df['away_probable_pitcher'][row]
        team_df['home_starter'] = game_df['home_probable_pitcher'][row]
        team_df['away_starter'] = team_df['away_starter'].apply(remove_accents)
        team_df['home_starter'] = team_df['home_starter'].apply(remove_accents)
        
        # Assign Leverage of 1 to starting pitcher
        team_df['Leverage'] = np.where((team_df['fullName'] == team_df['away_starter']) | (team_df['fullName'] == team_df['home_starter']), 1, team_df['Leverage'])

        # Determine batting order
        team_df['order'] = pd.to_numeric(team_df['order'], errors='coerce')
        team_df['batting_order'] = np.nan
        for i in range(9):
            team_df['batting_order'] = np.where(team_df['order'] == (i+1)*100, i+1, team_df['batting_order'])

        ### Batters
        batter_df = team_df[team_df['position'] != "Pitcher"]
        batter_df['date_time'] = pd.to_datetime(batter_df['date'], format='%Y%m%d')
        batter_df['date_time'].fillna(batter_df['date_time'].min(), inplace=True)
            
        ## Dataset
        # Merge in stats
        # # Merge LHP stats
        batter_df = pd.merge_asof(
            batter_df,
            vs_lhp[['id', 'date_time'] + batter_inputs + ['imp_b', 'pa_b', 'pa_b_long']],
            on='date_time',
            by='id',
            direction='backward',
            # suffixes=('', '_l')
        )

        # Merge RHP stats
        batter_df = pd.merge_asof(
            batter_df,
            vs_rhp[['id', 'date_time'] + batter_inputs + ['imp_b', 'pa_b', 'pa_b_long']],
            on='date_time',
            by='id',
            direction='backward',
            suffixes=('_l', '_r')
        )

        # Merge Steamer
        batter_df = pd.merge_asof(
            batter_df,
            steamer_hitters_df,
            on='date_time',
            by='id',
            direction='backward',
            suffixes=('', '_steamer')
        )

        # Remove redundant variables
        batter_df.drop(columns={'firstname', 'lastname', 'mlbamid', 'fullName2', 'position2'}, inplace=True)

        # Move 'batting_order' to the desired position
        batter_df.insert(batter_df.columns.get_loc('order') + 1, 'batting_order', batter_df.pop('batting_order'))
        
        # Sort
        batter_df['batting_order'] = pd.to_numeric(batter_df['batting_order'], errors='coerce')
        batter_df.sort_values('batting_order', inplace=True)


        ### Pitchers
        pitcher_df = team_df[(team_df['position'] == "Pitcher") | (team_df['position'] == "Two-Way Player")]
        pitcher_df['date_time'] = pd.to_datetime(pitcher_df['date'], format='%Y%m%d')
        pitcher_df['date_time'].fillna(pitcher_df['date_time'].min(), inplace=True)

        ## Dataset
        # Merge in stats
        # # Merge LHP stats
        pitcher_df = pd.merge_asof(
            pitcher_df,
            vs_lhb[['id', 'date_time'] + pitcher_inputs + ['imp_p', 'pa_p', 'pa_p_long']],
            on='date_time',
            by='id',
            direction='backward',
            # suffixes=('', '_l')
        )

        # Merge RHP stats
        pitcher_df = pd.merge_asof(
            pitcher_df,
            vs_rhb[['id', 'date_time'] + pitcher_inputs + ['imp_p', 'pa_p', 'pa_p_long']],
            on='date_time',
            by='id',
            direction='backward',
            suffixes=('_l', '_r')
        )

        # Merge Steamer
        pitcher_df = pd.merge_asof(
            pitcher_df,
            steamer_pitchers_df,
            on='date_time',
            by='id',
            direction='backward',
            suffixes=('', '_steamer')
        )

        # Remove redundant variables
        pitcher_df.drop(columns={'firstname', 'lastname', 'mlbamid', 'fullName2', 'position2'}, inplace=True)

        # Move 'batting_order' to the desired position
        pitcher_df.insert(pitcher_df.columns.get_loc('order') + 1, 'batting_order', pitcher_df.pop('batting_order'))

        # Sort
        pitcher_df['Leverage'] = pd.to_numeric(pitcher_df['Leverage'], errors='coerce')
        pitcher_df.sort_values('Leverage', inplace=True)

        # Assign team
        if team == away_team:
            away_batter_df = batter_df.copy()
            away_pitcher_df = pitcher_df.copy()
        else:
            home_batter_df = batter_df.copy()
            home_pitcher_df = pitcher_df.copy()

    # Drop duplicates: 
    away_batter_df.drop_duplicates('id', keep='last', inplace=True)
    away_batter_df.reset_index(drop=True, inplace=True)
    
    home_batter_df.drop_duplicates('id', keep='last', inplace=True)
    home_batter_df.reset_index(drop=True, inplace=True)
    
    away_pitcher_df.drop_duplicates('id', keep='last', inplace=True)
    away_pitcher_df.reset_index(drop=True, inplace=True)
    
    home_pitcher_df.drop_duplicates('id', keep='last', inplace=True)
    home_pitcher_df.reset_index(drop=True, inplace=True)
       
    return away_batter_df, away_pitcher_df, home_batter_df, home_pitcher_df

##### Create Matchup Files

In [ ]:
def create_matchup_files(game_df, row, vs_lhp, vs_rhp, vs_rhb, vs_lhb, steamer_hitters_df, steamer_pitchers_df, team_dict):
    # Extract IDs
    game_id = game_df['game_id'][row]
    away_id = game_df['away_id'][row]
    home_id = game_df['home_id'][row]

    # Retrieve Baseball Reference team abbreviation
    away_team = team_dict[away_id]
    home_team = team_dict[home_id]

    # Extract date
    game_date = game_df['game_date'][row]
    game_date = game_date.replace("-", "")
    game_datetime = game_df['game_datetime'][row]

    # Convert string to datetime object
    utc_datetime = datetime.datetime.strptime(game_datetime, "%Y-%m-%dT%H:%M:%SZ")

    # Define the UTC timezone
    utc_timezone = pytz.timezone("UTC")

    # Set the UTC timezone for the datetime object
    utc_datetime = utc_timezone.localize(utc_datetime)

    # Convert to Eastern Standard Time (EST)
    est_timezone = pytz.timezone("US/Eastern")
    est_datetime = utc_datetime.astimezone(est_timezone)

    # Format the result
    formatted_time = est_datetime.strftime("%H%M")

    
    # Create position dfs
    away_batter_df, away_pitcher_df, home_batter_df, home_pitcher_df = create_matchup_file(game_df, row, vs_lhp, vs_rhp, vs_rhb, vs_lhb, steamer_hitters_df, steamer_pitchers_df, team_dict)

    
    # Create folder, if it doesn't exist
    os.makedirs(os.path.join(baseball_path, "B01. Matchups", f'Matchups {game_date}'), exist_ok=True)

    # File name
    matchup_file = f"{away_team}@{home_team} {game_id} {formatted_time}"

    # Write to Excel
    away_batter_df.to_excel(os.path.join(baseball_path, "B01. Matchups", f'Matchups {game_date}', f'{matchup_file}.xlsx'), sheet_name="AwayBatters", engine='openpyxl', index=False)

    with pd.ExcelWriter(os.path.join(baseball_path, "B01. Matchups", f'Matchups {game_date}', f'{matchup_file}.xlsx'), mode='a', engine='openpyxl') as writer:  
        home_batter_df.to_excel(writer, sheet_name='HomeBatters', index=False)

    with pd.ExcelWriter(os.path.join(baseball_path, "B01. Matchups", f'Matchups {game_date}', f'{matchup_file}.xlsx'), mode='a', engine='openpyxl') as writer:  
        away_pitcher_df.to_excel(writer, sheet_name='AwayPitchers', index=False)

    with pd.ExcelWriter(os.path.join(baseball_path, "B01. Matchups", f'Matchups {game_date}', f'{matchup_file}.xlsx'), mode='a', engine='openpyxl') as writer:  
        home_pitcher_df.to_excel(writer, sheet_name='HomePitchers', index=False)

In [ ]:
game_df[['home_probable_pitcher', 'away_probable_pitcher']] = game_df[['home_probable_pitcher', 'away_probable_pitcher']].fillna("Missing Starter")

### Run

In [ ]:
%%time
print(len(game_df))
empty_list = Parallel(n_jobs=5, verbose=True)(delayed(create_matchup_files)(game_df, row, vs_lhp, vs_rhp, vs_rhb, vs_lhb, steamer_hitters_df, steamer_pitchers_df, team_dict) for row in range(len(game_df)))